# Selected Topics in Nature-inspired Algorithms

Group 5/ Jannik Zeiser, Caro Gass, AC Meisener, Shahd (Honey) Safarani, Kim Gerbaulet, Adrián Rojas

# Knapsack-Problem: 

given items I1, . . . , In with weights w1, . . . ,wn,
values v1, . . . , vn and a weight limit W , maximize value

### Nature-inspired Situation:
Given your student appartment is a mess and your parents / a sexy friend / many sexy friends 
are/is coming over for a visit in 2 hours. You would need much longer to clean and tidy up the whole situation. 
Thus, you need to consider your priorities and calculate the gain e.g what makes the best impression if its clean, where does the visitor look at or which room he/she/ they will enter.

# Weights and Actions

### weight limit0 "W": 
2 hours or 120 minutes

### items "I" or what's to do: 

|     Action i        |        Weight w      | Value v|
|---------------------|:--------------------:|-------:|
|make the bed         |      2 min           |   4    |
|do the dishes        |     10 min           |   6    |
|vacuum               |      7 min           |   8    |
|swipe the floors     |     20 min           |  11    |
|fold your clothes    |      8 min           |  10    |
|dusting the shelves  |     15 min           |   5    |
|clean the kitchen    |     10 min           |   6    |
|clean the toilet     |      4 min           |   4    |
|clean the shower     |      4 min           |   2    |
|clean the basin      |      4 min           |   8    |
|clean the sink       |      4 min           |   2    |
|bring out the rubbish|      6 min           |   4    |
|clean the windows    |     60 min           |  10    |
|clean mirrows        |      5 min           |   2    |
|shower & get dress   |     20 min           |  20    |
|brush your teeth     |      3 min           |   2    |

# The neighbourhoods

For our neighbourhood conditions we make the following assumption:
*0/1 neighbour* - we expand our neighbourhood by randomly dropping or adding an item *0/1 neighbour*

###  The small neighbourhood: 
#### swap neighbourhood + *0/1 neighbour* and its swaps
our neighbourhood is the set of the swap neighbourhood of 
the current state, the *0/1 neighbour* and its swap neighbourhood


###  The large neighbourhood: 
#### transposition neighbourhood + 0/1 and its transpositions
our neighbourhood is the set of the transposition neighbourhood 
of the current state, the *0/1 neighbour* and its transposition neighbourhood



In [14]:
import numpy as np
import pandas as pd
import random as rd
import itertools

# Weight limit
W = 120

items = ["make the bed", "do the dishes", "vacuum", "swipe the floors", 
         "fold your clothes", "dusting the shelves", "clean the kitchen", 
         "clean the toilet", "clean the shower", "clean the basin", 
         "clean the sink", "bring out the rubbish", "clean the windows", 
         "clean mirrows", "shower & get dress", "brush your teeth"]
weights = np.array([2, 10, 7, 20, 8, 15, 10, 4, 4, 4, 4, 6, 60, 5, 20, 3])
values =  np.array([4, 6, 8, 11, 10, 5, 6, 4, 2, 8, 2, 4, 10, 2, 20, 2])



In [15]:
# This cell we finally didn't even use

from random import shuffle 

cleaning_list = pd.DataFrame(columns=['action','weight','value'])

cleaning_list['action'] = items
cleaning_list['weight'] = weights
cleaning_list['value']  = values

# random_assignment for Knapsack - the starting assignment
# will be above our weight limit thus we have to stop "cleaning" 
# as soon as we run out of time.
schedule = shuffle(items)

# the stop cleaning function  - not used in the end
def time_over(current_schedule, W):
    performance = 0
    for i in range(0,len(schedule)):
        value = cleaning_list['value'][cleaning_list['action' == schedule[i]]]
        if (performance + value) > W:
            final_schedule = schedule[0:i-1]
            return final_schedule, performance
    return schedule, performance

In [16]:
# Calculates the gain of an assignment x
def gain(x):
    return np.sum(x*values)

# Checks for exceeding the weight limit
def above_weight_limit(x):
    return np.sum(x*weights)>W

# Swaps between two elements and creates a copy of the list
def swap(x, idx1, idx2):
    l = x.copy()
    l[idx1], l[idx2] = l[idx2], l[idx1]
    return l

#Initialize a feasible random assignment
def init():
    init_assign = [rd.choice([0,1]) for i in range(len(weights))]
    while above_weight_limit(init_assign):
        init_assign = [rd.choice([0,1]) for i in range(len(weights))]
    return init_assign

# Brute Force (neighbourhood = complete set)
def brtfrc(x):
    powerset = []
    for i in itertools.product([0,1], repeat = len(x)):
        powerset.append(i)
    return powerset
# Creates a to-do-list corresponding to a 0/1-list
def todo(a):
    b = []
    a1 = a.copy()
    for i in a1: 
        if i == 1:
            b.append(items[a1.index(i)])
            a1[a1.index(i)] = 0
    return b 

In [17]:
#Creates transposition neighbourhood for x
def transposes(x):
    #neighbours of same number of selected objects
    a = [ swap(x, i, j) for i in range(len(x)) for j in range(i+1, len(x)) ]
    #Adding or removing an object randomly and getting the neighbors
    idx = rd.choice(range(0, len(x)))
    x1 = x.copy()
    x1[idx] = 0 if x1[idx]==1 else 1
    a1 = [ swap(x1, i, j) for i in range(len(x1)) for j in range(i+1, len(x1)) ]
    #Merging both neighborhoods
    a = [j for i in zip(a,a1) for j in i]
    return  list( list(x) for x in set(tuple(i) for i in a if i!=x) )

In [18]:
#Creates swap neighbourhood for x
def swaps(x):
    #neighbours of same number of selected objects
    a = [ swap(x, i, (i+1)%len(x)) for i in range(len(x)) ] 
    #Adding or removing an object randomly and getting the neighbors
    idx = rd.choice(range(0, len(x)))  #
    x1 = x.copy()
    x1[idx] = 0 if x1[idx]==1 else 1
    a1 = [ swap(x1, i, (i+1)%len(x1)) for i in range(len(x1)) ]
    #Merging both neighborhoods
    a = [j for i in zip(a,a1) for j in i]
    return  list( list(x) for x in set(tuple(i) for i in a if i!=x) )

In [19]:
# Hill_Climbing swap neigbourhood
def hc_s(a=None):
    if not a:
        a = init()  # generate initial assignment if non is given
    steps = 0  # iteration counter
    
    while True:
        steps += 1  # count iterations
        neighbours = swaps(a) # create full swap neighbourhood
        neighbours = [n for n in neighbours if not above_weight_limit(n)]  # collect only neigbours below the weight limit
        if not neighbours: break  # neighbours are empty if all neighbours are above weight limit
        gains = [gain(nbr) for nbr in neighbours]  # the value gains of our neighbours
        if max(gains) <= gain(a): break # breaks if all gains are smaller than the current gain
        a = neighbours[gains.index(max(gains))]  # best neighbour becomes new state
    return a, gain(a), steps  # solution, optimal preformance/gain we get, iterations

In [20]:
# Hill_Climbing transposition neighbourhood
def hc_t(a=None):
    if not a:
        a = init()  # generate initial assignment if non is given
    steps = 0
    
    while True:
        steps += 1
        neighbours = transposes(a) # create full transposition neighbourhood
        neighbours = [n for n in neighbours if not above_weight_limit(n)]
        if not neighbours: break
        gains = [gain(nbr) for nbr in neighbours]
        if max(gains) <= gain(a): break
        a = neighbours[gains.index(max(gains))]
    return a, gain(a), steps  # solution, optimal performance/gain we get, iterations

In [21]:
## First Choice Hill_Climbing

# First Choice Hill_Climbing swap
def fchc_s(a=None):
    if not a:
        a = init()  # generate initial assignment if non is given  
    steps = 0
    while True:
        current_sol = a.copy()
        current_gain = gain(a)
        steps += 1
        neighbours = swaps(a) #swap or transposition
        neighbours = [n for n in neighbours if not above_weight_limit(n)]
        if not neighbours: break
        for item in neighbours:
            if gain(item) > current_gain:
                a = item  #solution, optimal cost we got, iterations
                break
        if current_sol==a : break
    return a, gain(a), steps

In [22]:
# First Choice Hill_Climbing transpose
def fchc_t(a=None):
    if not a:
        a = init() # generate initial assignment if non is given
    steps = 0  # iteration counter
    
    while True:
        current_sol = a.copy()
        current_gain = gain(a)
        steps += 1  # count iterations
        neighbours = transposes(a) # swap or transposition
        neighbours = [n for n in neighbours if not above_weight_limit(n)]
        if not neighbours: break
        for item in neighbours:
            if gain(item) > current_gain:
                a = item  #solution, optimal cost we got, iterations
                break
        if current_sol==a : break
    return a, gain(a), steps  #solution, optimal performance we get, iterations

In [23]:
#Hill_Climbing Whole Set

def hc_ws(a=None):
    if not a:
        a = init()  # generate initial assignment if non is given
    steps = 0
    
    while True:
        steps += 1
        neighbours = brtfrc(a) # create whole set neighbourhood
        neighbours = [n for n in neighbours if not above_weight_limit(n)]
        if not neighbours: break
        gains = [gain(nbr) for nbr in neighbours]
        if max(gains) <= gain(a): break
        a = neighbours[gains.index(max(gains))]
    return a, gain(a), steps  # solution, optimal performance/gain we get, iterations

In [24]:
#Number of test iterations
Num_test = 100

perf_hc_s = 0
perf_hc_t = 0
perf_fchc_s = 0
perf_fchc_t = 0

steps_hc_s = 0
steps_hc_t = 0
steps_fchc_s = 0
steps_fchc_t = 0

In [25]:
for i in range (0,Num_test):
    a = init()
    perf_hc_s = perf_hc_s + hc_s(a)[1]
    perf_hc_t = perf_hc_t + hc_t(a)[1]
    perf_fchc_s = perf_fchc_s + fchc_s(a)[1]
    perf_fchc_t = perf_fchc_t + fchc_t(a)[1]
    
    steps_hc_s = steps_hc_s + hc_s(a)[2]
    steps_hc_t = steps_hc_t + hc_t(a)[2]
    steps_fchc_s = steps_fchc_s + fchc_s(a)[2]
    steps_fchc_t = steps_fchc_t + fchc_t(a)[2]

print("Iterations:", Num_test)
print("HC with swap | Average performance:", perf_hc_s/Num_test, "| Average steps:", steps_hc_s/Num_test)
print("HC with transposition | Average performance:", perf_hc_t/Num_test,"| Average steps:", steps_hc_t/Num_test)
print("First-Choice HC with swap | Average performance:", perf_fchc_s/Num_test,"| Average steps:", steps_fchc_s/Num_test)
print("First-Choice HC with transposition | Average performance:", perf_fchc_t/Num_test,"| Average steps:", steps_fchc_t/Num_test)
lwhole, gainwhole, stepswhole = hc_ws(a)
print("First-Choice Hill-Climbing with whole set neighbourhood:", lwhole, gainwhole, stepswhole)
print(todo(list(lwhole)))

Iterations: 100
HC with swap | Average performance: 73.71 | Average steps: 4.87
HC with transposition | Average performance: 80.93 | Average steps: 5.36
First-Choice HC with swap | Average performance: 71.86 | Average steps: 7.1
First-Choice HC with transposition | Average performance: 82.51 | Average steps: 10.91
First-Choice Hill-Climbing with whole set neighbourhood: (1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1) 92 2
['clean the toilet', 'vacuum', 'clean mirrows', 'dusting the shelves', 'fold your clothes', 'swipe the floors', 'clean the sink', 'bring out the rubbish', 'clean the kitchen', 'clean the basin', 'brush your teeth', 'shower & get dress', 'do the dishes', 'clean the shower']
